In [21]:
import numpy as np
import math
import os
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from enum import Enum
from collections import Counter
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leechilvers/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leechilvers/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leechilvers/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Get the positive and negative reviews
def get_reviews_in(path):
    review_file_pattern = re.compile('\d+_\d+.txt')
    files = os.listdir(path)
    # get the review file names
    review_files = [file for file in files if review_file_pattern.match(file)] 
    # get the content of each review file
    reviews = []
    for review_file in review_files:
        review_file_path = os.path.join(path, review_file)
        with open(review_file_path, 'r') as content:
            reviews.append(content.read())

    return reviews

# x data
pos_reviews = get_reviews_in('film_reviews/pos')
neg_reviews = get_reviews_in('film_reviews/neg')
reviews = pos_reviews + neg_reviews

# y data
pos_labels = [0] * len(pos_reviews)
neg_labels = [1] * len(neg_reviews)
labels = pos_labels + neg_labels

Data Splits

In [5]:
# split the data into train/dev/test splits
# 80% train and 20% test
x_train, x_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=42)
# split test into 50% dev and 50% test
x_dev, x_test, y_dev, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [24]:
class NormTech(Enum):
    TF_IDF = 1
    FREQ_NORM = 2
    PPMI = 3

class MyFeatureSelector:
    
    def selectFeatures(self, reviews, normalisation_technique = NormTech.TF_IDF):
         # [[('this', 'is', 'a'), ('is', 'a', 'review'), ('a', 'review', None)]]

         # feature selection
        if (normalisation_technique == NormTech.TF_IDF):
            reviews = self.__tf_idf(reviews)
        elif (normalisation_technique == NormTech.FREQ_NORM):
            reviews = self.__freq_norm(reviews)
        elif (normalisation_technique == NormTech.PPMI):
            reviews = self.__ppmi(reviews)
            
        return reviews
    
    def __tf_idf(self, reviews):

        def __get_tf(term, review):
            # tf = number of occurences of the term in the review / number of terms in the review
            # TODO: weird object type preventing arithmetic
            counter = Counter(review)
            print(counter.values)
            term_occurances_count = counter[term]
            total_terms_count = sum(counter.values)
            
            return term_occurances_count/total_terms_count
        
        def __get_idf(term):
            # idf = log(number of reviews / number of reviews that contain the term)
            reviews_count = len(reviews)
            reviews_with_term_count = 1
            for review in reviews:
                if term in review: reviews_with_term_count += 1
            
            return math.log(reviews_count/reviews_with_term_count)
        
        # set (no repeats) of all terms across all documents
        terms = set([term for review in reviews for term in review])

        # is {term1: [score1, score2], term2: [score1, score2], ...}
        tf_map = {term: [__get_tf(term, review) for review in reviews] for term in terms }
        # is {term1: score1, term2: score2, ...}
        idf_map = {term: __get_idf(term) for term in terms}
        # is [{term1: score1, term2: score2, ...}, {term1: score1, term2: score2, ...}]
        tf_idf_map = [{term: (tf_map[term][i] * idf_map[term]) for term in terms } for i in range(len(reviews))]

        filtered_reviews = []
        for i, review_tf_idfs in enumerate(tf_idf_map):
            # get the most useful features in a list: [('a', 'useful', 'feature'), ('useful', 'feature', None)]
            selected_features = [feature for feature, score in review_tf_idfs.items() if score > 0.01] #TODO: define condition
            # modify review so it only has the useful features
            filtered_review = [feature for feature in reviews[i] if feature in selected_features]
            filtered_reviews.append(filtered_review)
        
        return filtered_reviews
    
    def __freq_norm(self, reviews):
        return reviews
    
    def __ppmi(self, reviews):
        return reviews

In [25]:

class MyFeatureGenerator:

    def generateFeatures(self, reviews, 
                         is_lemmatisation = True, 
                         is_lowercase = True, 
                         is_stopwords_removed = True,
                         is_punctuation_removed = False, 
                         n_gram_len = 1, 
                         normalisation_technique = NormTech.TF_IDF
                         ):
        # tokenise each review: ['this', 'is', 'a', 'review']
        reviews = self.__tokenise(reviews, is_punctuation_removed, is_stopwords_removed, is_lowercase)

        # do lemmatisation or stemming: ['this', 'is', 'a', 'review']
        reviews = self.__lemmatise(reviews) if is_lemmatisation else self.__stem(reviews)

        # generate n_grams: [[('this', 'is', 'a'), ('is', 'a', 'review'), ('a', 'review', None)]]
        reviews = self.__n_gram(reviews, n_gram_len)

        # feature selection
        feature_selector = MyFeatureSelector()
        reviews = feature_selector.selectFeatures(reviews, normalisation_technique)

        return reviews

    def __tokenise(self, reviews, is_punctuation_removed, is_stopwords_removed, is_lowercase):
        tokenised_reviews = []
        # tokenise each review using nltk
        remove_punc_tokeniser = nltk.RegexpTokenizer('\w+')
        stop_words = set(stopwords.words('english'))

        for review in reviews:
            # generate tokens with/out punctuation 
            tokens = remove_punc_tokeniser.tokenize(review) if (is_punctuation_removed) else nltk.word_tokenize(review)
            tokenised_review = []
            for token in tokens:
                # ignoring stopwords
                if (is_stopwords_removed and token.lower() in stop_words):
                        continue
                
                # converting to lowercase
                result_token = token.lower() if (is_lowercase) else token
                
                tokenised_review.append(result_token)
            tokenised_reviews.append(tokenised_review)
        
        return tokenised_reviews
                        
    def __lemmatise(self, reviews):
        lemmatised_reviews = []
        # do lemmatisation on tokenised reviews
        lemmatiser = nltk.WordNetLemmatizer()
        for review in reviews:
            # review is ['a', 'list', 'of', 'words']
            lemmatised_reviews.append([lemmatiser.lemmatize(token) for token in review])
        
        return lemmatised_reviews
    
    def __stem(self, reviews):
        stemmed_reviews = []
        # do stemming on tokenised reviews
        stemmer = nltk.PorterStemmer()
        for review in reviews:
            # review is ['a', 'list', 'of', 'words']
            stemmed_reviews.append([stemmer.stem(token) for token in review])
        
        return stemmed_reviews
    
    def __n_gram(self, reviews, n):
        # default to 1 if not valid length
        if (n < 1): n = 1
        # generate n-gram on processed reviews where
        # a review is ['a', 'list', 'of', 'words']
        return [nltk.ngrams(review, n) for review in reviews]

feature_generator = MyFeatureGenerator()
feature_generator.generateFeatures(x_train, n_gram_len=3)

<built-in method values of Counter object at 0x7fa53678df90>


TypeError: 'builtin_function_or_method' object is not iterable

Naive Bayes

In [7]:
""" evaluate at least three feature sets with classifier """

# TODO: CHECK MY FEATURES WORK FOR STANDARD NB IMPLEMENTATION

class MyNaiveBayesClassifier:
    # for sentiment analysis there will be 3 classes
    class_count = 0

    def get_prior_probability(self, labels):
        # movie review labels is [0(pos), 1(neg), 2(neu), 0, 2, 0, ...]
        # array of p(class) where index is class
        prior_probs = np.zeros(max(labels) + 1)
        self.class_count = len(prior_probs)
        for label in labels:
            prior_probs[label] += 1/len(labels) #TODO: log(1/len(labels))
        
        return prior_probs